rank\
Team (constructor name)\
Wins\
Points

In [0]:
dbutils.widgets.text('p_file_date', "2021-03-21")
v_file_date = dbutils.widgets.get('p_file_date')

In [0]:
%run ../includes/configuration

In [0]:
%run ../includes/common_functions

In [0]:
from pyspark.sql.functions import col, count, sum, when, desc, rank
from pyspark.sql.window import Window

Find race years for which data should be processed

In [0]:
race_results_list = spark.read.format("delta").load(f"{presentation_folder_path}/f1_presentation/race_results")\
    .filter(f"file_date = '{v_file_date}'")

In [0]:
race_year_list = df_col_to_list(race_results_list, 'race_year')

In [0]:
race_results_df = spark.read.format("delta").load(f"{presentation_folder_path}/f1_presentation/race_results")\
     .filter(col("race_year").isin(race_year_list))

In [0]:
constructor_standings_df = race_results_df\
    .groupBy("race_year", "team")\
        .agg(sum("points").alias("total_points")\
            ,count(when(col("position") == 1, True)).alias("Wins"))\
                .orderBy(desc("total_points"))

In [0]:
constructorRankSpec = Window.partitionBy("race_year").orderBy(desc("total_Points"), desc("wins"))

In [0]:
final_df = constructor_standings_df\
    .withColumn("Rank", rank().over(constructorRankSpec))

In [0]:
#final_df.write\
#    .mode("overwrite")\
#        .parquet(f"{presentation_folder_path}/constructor_standings")

#final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.constructor_standings")

#incremental_load(final_df, 'race_year', 'f1_presentation', 'constructor_standings')

In [0]:
merge_condition = "tgt.race_year == src.race_year and tgt.team = src.team"

merge_delta_table(final_df, 'f1_presentation', 'constructor_standings', presentation_folder_path, merge_condition, 'race_year')

In [0]:
%sql
SELECT race_year, COUNT(race_year)
FROM f1_presentation.constructor_standings
GROUP BY race_year
ORDER BY race_year DESC;

race_year,count(race_year)
2021,10
2020,10
2019,10
2018,10
2017,10
2016,11
2015,10
2014,11
2013,11
2012,12


In [0]:
%sql
SELECT *
FROM f1_presentation.constructor_standings
WHERE race_year = '2021'

race_year,team,total_points,Wins,Rank
2021,Mercedes,60.0,1,1
2021,Red Bull,53.0,1,2
2021,McLaren,41.0,0,3
2021,Ferrari,34.0,0,4
2021,AlphaTauri,8.0,0,5
2021,Aston Martin,5.0,0,6
2021,Alpine F1 Team,3.0,0,7
2021,Alfa Romeo,0.0,0,8
2021,Williams,0.0,0,8
2021,Haas F1 Team,0.0,0,8
